# Test PySpark Cluster
Questo notebook testa le funzionalità principali del cluster Spark con supporto per Delta Lake, Iceberg e Hudi.

## 1. Inizializzazione Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max, min
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

spark = SparkSession.builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

print("✅ Spark Session già attiva")
print(f"Spark Version: {spark.version}")
print(f"App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/02 23:29:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/02 23:29:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ Spark Session già attiva
Spark Version: 4.0.1
App Name: Spark-Notebook
Master: spark://spark-master:7077


## 2. Test Base DataFrame Operations

In [ ]:
# Creazione dati di esempio
data = [
    ("Alice", 25, 85000.0, "Engineering"),
    ("Bob", 30, 75000.0, "Marketing"),
    ("Carol", 35, 95000.0, "Engineering"),
    ("David", 28, 65000.0, "Sales"),
    ("Eve", 32, 88000.0, "Engineering"),
    ("Frank", 27, 72000.0, "Marketing")
]

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", DoubleType(), True),
    StructField("department", StringType(), True)
])

df = spark.createDataFrame(data, schema)
print("📊 DataFrame creato")
df.show()

📊 DataFrame creato


25/12/02 23:30:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/02 23:30:17 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/02 23:30:32 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/02 23:30:50 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [3]:
# Analisi statistiche
print("📈 Statistiche per dipartimento:")
dept_stats = df.groupBy("department").agg(
    count("*").alias("count"),
    avg("age").alias("avg_age"),
    avg("salary").alias("avg_salary"),
    max("salary").alias("max_salary")
)
dept_stats.show()

📈 Statistiche per dipartimento:


[Stage 3:>                                                        (0 + 14) / 14]

+-----------+-----+------------------+-----------------+----------+
| department|count|           avg_age|       avg_salary|max_salary|
+-----------+-----+------------------+-----------------+----------+
|Engineering|    3|30.666666666666668|89333.33333333333|   95000.0|
|  Marketing|    2|              28.5|          73500.0|   75000.0|
|      Sales|    1|              28.0|          65000.0|   65000.0|
+-----------+-----+------------------+-----------------+----------+



## 3. Test Performance e Parallelismo

In [16]:
# Crea un dataset più grande per testare il parallelismo
import random

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
  StructField("name", StringType(), True),
  StructField("age", IntegerType(), True),
  StructField("salary", DoubleType(), True),  # Usa DoubleType invece di IntegerType
  StructField("department", StringType(), True)
])

large_data = []
departments = ["Engineering", "Marketing", "Sales", "HR", "Finance"]
names = ["Alice", "Bob", "Carol", "David", "Eve", "Frank", "Grace", "Henry", "Ivy", "Jack"]

for i in range(10000):
    name = f"{random.choice(names)}_{i}"
    age = random.randint(22, 65)
    salary = float(random.randint(40000, 150000))
    dept = random.choice(departments)
    large_data.append((name, age, salary, dept))

large_df = spark.createDataFrame(large_data, schema)
print(f"📊 Dataset grande creato: {large_df.count()} righe")

# Test aggregazione su dataset grande
print("⚡ Test performance aggregazione:")
result = large_df.groupBy("department").agg(
    count("*").alias("employees"),
    avg("salary").alias("avg_salary")
).orderBy("avg_salary", ascending=False)

result.show()

📊 Dataset grande creato: 10000 righe
⚡ Test performance aggregazione:
+-----------+---------+-----------------+
| department|employees|       avg_salary|
+-----------+---------+-----------------+
|      Sales|     2021|95769.59574468085|
|Engineering|     1942| 94423.8455200824|
|    Finance|     2056|94232.65758754865|
|  Marketing|     2003|94030.96105841239|
|         HR|     1978|93188.38877654196|
+-----------+---------+-----------------+



## 5. Test SQL Interface

In [17]:
# Registra il DataFrame come tabella temporanea
large_df.createOrReplaceTempView("employees")

# Esegui query SQL
sql_result = spark.sql("""
    SELECT 
        department,
        COUNT(*) as employee_count,
        ROUND(AVG(salary), 2) as avg_salary,
        MIN(salary) as min_salary,
        MAX(salary) as max_salary
    FROM employees 
    WHERE age >= 25 AND age <= 45
    GROUP BY department
    ORDER BY avg_salary DESC
""")

print("🔍 Query SQL - Dipendenti 25-45 anni:")
sql_result.show()

🔍 Query SQL - Dipendenti 25-45 anni:
+-----------+--------------+----------+----------+----------+
| department|employee_count|avg_salary|min_salary|max_salary|
+-----------+--------------+----------+----------+----------+
|      Sales|           971|  95298.68|   40005.0|  149882.0|
|    Finance|          1011|  94414.19|   40332.0|  149955.0|
|         HR|           958|  94209.39|   40142.0|  149939.0|
|Engineering|           910|  93503.43|   40244.0|  149946.0|
|  Marketing|           975|  93175.97|   40069.0|  149802.0|
+-----------+--------------+----------+----------+----------+



## 6. Test Cluster Info

In [21]:
# Informazioni sul cluster
sc = spark.sparkContext

print("🖥️ Informazioni Cluster:")
print(f"Default Parallelism: {sc.defaultParallelism}")
print(f"Application ID: {sc.applicationId}")
print(f"Spark Version: {sc.version}")
print(f"Python Version: {sc.pythonVer}")

🖥️ Informazioni Cluster:
Default Parallelism: 14
Application ID: app-20251202134751-0000
Spark Version: 3.5.7
Python Version: 3.12


## 7. Cleanup

In [19]:
# Pulizia tabelle temporanee
spark.catalog.dropTempView("employees")
print("🧹 Tabelle temporanee rimosse")
print("✅ Test completato con successo!")

🧹 Tabelle temporanee rimosse
✅ Test completato con successo!
